In [1]:
from SeqEN2.utils.data_loader import read_json, write_json
from pathlib import Path
from IPython.display import clear_output
from glob import glob
from os import system

In [2]:
import numpy as np
import pandas as pd

In [15]:
import prody

In [24]:
new_wd = Path('../../data/pdb_dssp_phipsi_ndxed/')
if not new_wd.exists():
    new_wd.mkdir()
pdb_files = Path('../../data/pdb_files/')
if not pdb_files.exists():
    pdb_files.mkdir()

In [4]:
data_dir = '../../data/pdb_dssp_phipsi/'
files = sorted(glob(f'{data_dir}*.json.gz'))

aa_dict = {
    "A": "ALA", "C": "CYS", "D": "ASP", "E": "GLU", "F": "PHE", "G": "GLY", "H": "HIS", "I": "ILE", "K": "LYS", "L": "LEU", 
    "M": "MET", "N": "ASN", "P": "PRO", "Q": "GLN", "R": "ARG", "S": "SER", "T": "THR", "V": "VAL", "W": "TRP", "Y": "TYR"
}


In [32]:
def get_psiphi(d, aa, i): 
    try:
        return d[f"{aa_dict[aa]}-{i+1}"]
    except KeyError:
        return [np.nan, np.nan]
    
dropped_seq = {}
        
for file in files[:]:
    print(f'transforming {file}')
    filename = file.split('/')[-1]
    data = read_json(file)
    keys = list(data.keys())
    new_data = {}
    aa_keys = "WYFMILVAGPSTCEDQNHRK*"
    ss_keys = "LSTIGHBE*"
    w = 20
    padding = w - 1
    for key in keys:
        item = data[key]
        if len(item["Sequence"]) < 40:
            dropped_seq[key] = {"item": item, "error": "shorter than 40 aa"}
            continue
        seq = ("*" * padding) + item["Sequence"] + ("*" * padding)
        ss_seq = ("*" * padding) + item["DSSP"] + ("*" * padding)
        assert len(seq) == len(ss_seq), f"{key} failed to transform, seq and ss have different lengths."
        try:
            values = np.array([aa_keys.index(aa) for aa in seq], dtype=np.int8).reshape((-1, 1))
            values = np.append(values, np.array([ss_keys.index(ss) for ss in ss_seq], dtype=np.int8).reshape((-1, 1)), axis=1)
            values = np.append(values, np.array([get_psiphi(item["PsiPhi"], aa, i-padding) for i, aa in enumerate(seq)]).reshape((-1, 2)), axis=1)
            new_data[key] = values
        except ValueError:
            dropped_seq[key] = {"item": item, "error": "non-cononical aa"}
            continue
    write_json(new_data, str(new_wd / filename), encoder='numpy')
    
print(dropped_seq.keys())

transforming ../../data/pdb_dssp_phipsi/pdb_dssp_phipsi_0.json.gz
transforming ../../data/pdb_dssp_phipsi/pdb_dssp_phipsi_1.json.gz
transforming ../../data/pdb_dssp_phipsi/pdb_dssp_phipsi_10.json.gz
transforming ../../data/pdb_dssp_phipsi/pdb_dssp_phipsi_100.json.gz
transforming ../../data/pdb_dssp_phipsi/pdb_dssp_phipsi_101.json.gz
transforming ../../data/pdb_dssp_phipsi/pdb_dssp_phipsi_102.json.gz
transforming ../../data/pdb_dssp_phipsi/pdb_dssp_phipsi_103.json.gz
transforming ../../data/pdb_dssp_phipsi/pdb_dssp_phipsi_104.json.gz
transforming ../../data/pdb_dssp_phipsi/pdb_dssp_phipsi_105.json.gz
transforming ../../data/pdb_dssp_phipsi/pdb_dssp_phipsi_106.json.gz
transforming ../../data/pdb_dssp_phipsi/pdb_dssp_phipsi_107.json.gz
transforming ../../data/pdb_dssp_phipsi/pdb_dssp_phipsi_108.json.gz
transforming ../../data/pdb_dssp_phipsi/pdb_dssp_phipsi_109.json.gz
transforming ../../data/pdb_dssp_phipsi/pdb_dssp_phipsi_11.json.gz
transforming ../../data/pdb_dssp_phipsi/pdb_dssp_phips

In [61]:
print(len(dropped_seq.keys()))

11205


In [62]:
try_again = [key for key, item in dropped_seq.items() if item['error'] == "non-cononical aa"]

In [63]:
len(try_again)

1309

In [64]:
dropped_seq_2 = {}
aa_keys = "WYFMILVAGPSTCEDQNHRK*"
ss_keys = "LSTIGHBE*"
w = 20
padding = w - 1
new_data = {}
for key in try_again:
    seq = dropped_seq[key]['item']['Sequence']
    striped_seq = seq.strip('X')
    if 'X' not in striped_seq:
        leading_x = len(seq) - len(seq.lstrip('X'))
        leading_x = None if leading_x == 0 else leading_x
        trailing_x = len(seq) - len(seq.rstrip('X'))
        trailing_x = None if trailing_x == 0 else -trailing_x
        if leading_x or trailing_x:
            striped_ss = dropped_seq[key]['item']['DSSP'][leading_x:trailing_x]
            if len(striped_seq) == len(striped_ss):
                seq = ("*" * padding) + striped_seq + ("*" * padding)
                ss_seq = ("*" * padding) + striped_ss + ("*" * padding)
                try:
                    values = np.array([aa_keys.index(aa) for aa in seq], dtype=np.int8).reshape((-1, 1))
                    values = np.append(values, np.array([ss_keys.index(ss) for ss in ss_seq], dtype=np.int8).reshape((-1, 1)), axis=1)
                    values = np.append(values, np.array([get_psiphi(item["PsiPhi"], aa, i-padding) for i, aa in enumerate(seq)]).reshape((-1, 2)), axis=1)
                    new_data[key] = values
                except ValueError:
                    dropped_seq_2[key] = {"item": dropped_seq[key]['item'], "error": "non-cononical aa"}
                    continue
            else:
                dropped_seq_2[key] = {"item": dropped_seq[key]['item'], "error": "trim not working"}
        
    else:
        dropped_seq_2[key] = {"item": dropped_seq[key]['item'], "error": "internal X"}
write_json(new_data, str(new_wd / "leading_trailing_X_fixed.json.gz"), encoder='numpy')

In [65]:
print(len(dropped_seq_2.keys()))

392


In [66]:
try_again = [key for key, item in dropped_seq_2.items() if item['error'] == "internal X"]

In [67]:
dropped_seq_3 = {}
aa_keys = "WYFMILVAGPSTCEDQNHRK*"
ss_keys = "LSTIGHBE*"
w = 20
padding = w - 1
new_data = {}
for key in try_again:
    seq = dropped_seq_2[key]['item']['Sequence']
    striped_seq = seq.strip('X')
    if len(striped_seq) - len(striped_seq.replace('X', '')) == 1:
        striped_seq = seq.strip('X').replace('X', 'A')
        leading_x = len(seq) - len(seq.lstrip('X'))
        leading_x = None if leading_x == 0 else leading_x
        trailing_x = len(seq) - len(seq.rstrip('X'))
        trailing_x = None if trailing_x == 0 else -trailing_x
        if leading_x or trailing_x:
            striped_ss = dropped_seq_2[key]['item']['DSSP'][leading_x:trailing_x]
            if len(striped_seq) == len(striped_ss):
                seq = ("*" * padding) + striped_seq + ("*" * padding)
                ss_seq = ("*" * padding) + striped_ss + ("*" * padding)
                try:
                    values = np.array([aa_keys.index(aa) for aa in seq], dtype=np.int8).reshape((-1, 1))
                    values = np.append(values, np.array([ss_keys.index(ss) for ss in ss_seq], dtype=np.int8).reshape((-1, 1)), axis=1)
                    values = np.append(values, np.array([get_psiphi(item["PsiPhi"], aa, i-padding) for i, aa in enumerate(seq)]).reshape((-1, 2)), axis=1)
                    new_data[key] = values
                except ValueError:
                    dropped_seq_3[key] = {"item": dropped_seq_2[key]['item'], "error": "non-cononical aa"}
                    continue
        
    else:
        dropped_seq_3[key] = {"item": dropped_seq_2[key]['item'], "error": "internal X more than 1"}
write_json(new_data, str(new_wd / "fixed_with_one_internal_X.json.gz"), encoder='numpy')

In [71]:
print(len(dropped_seq_3.keys()))

113


In [73]:
try_again = [key for key, item in dropped_seq_3.items() if item['error'] == "internal X more than 1"]

113

In [74]:
dropped_seq_4 = {}
aa_keys = "WYFMILVAGPSTCEDQNHRK*"
ss_keys = "LSTIGHBE*"
w = 20
padding = w - 1
new_data = {}
for key in try_again:
    seq = dropped_seq_3[key]['item']['Sequence']
    striped_seq = seq.strip('X')
    if len(striped_seq) - len(striped_seq.replace('X', '')) == 2:
        striped_seq = seq.strip('X').replace('X', 'A')
        leading_x = len(seq) - len(seq.lstrip('X'))
        leading_x = None if leading_x == 0 else leading_x
        trailing_x = len(seq) - len(seq.rstrip('X'))
        trailing_x = None if trailing_x == 0 else -trailing_x
        if leading_x or trailing_x:
            striped_ss = dropped_seq_3[key]['item']['DSSP'][leading_x:trailing_x]
            if len(striped_seq) == len(striped_ss):
                seq = ("*" * padding) + striped_seq + ("*" * padding)
                ss_seq = ("*" * padding) + striped_ss + ("*" * padding)
                try:
                    values = np.array([aa_keys.index(aa) for aa in seq], dtype=np.int8).reshape((-1, 1))
                    values = np.append(values, np.array([ss_keys.index(ss) for ss in ss_seq], dtype=np.int8).reshape((-1, 1)), axis=1)
                    values = np.append(values, np.array([get_psiphi(item["PsiPhi"], aa, i-padding) for i, aa in enumerate(seq)]).reshape((-1, 2)), axis=1)
                    new_data[key] = values
                except ValueError:
                    dropped_seq_4[key] = {"item": dropped_seq_3[key]['item'], "error": "non-cononical aa"}
                    continue
        
    else:
        dropped_seq_4[key] = {"item": dropped_seq_3[key]['item'], "error": "internal X more than 2"}
write_json(new_data, str(new_wd / "fixed_with_two_internal_X.json.gz"), encoder='numpy')

In [75]:
print(len(dropped_seq_4.keys()))

94


In [76]:
try_again = [key for key, item in dropped_seq_4.items() if item['error'] == "internal X more than 2"]
len(try_again)

94

In [77]:
dropped_seq_5 = {}
aa_keys = "WYFMILVAGPSTCEDQNHRK*"
ss_keys = "LSTIGHBE*"
w = 20
padding = w - 1
new_data = {}
for key in try_again:
    seq = dropped_seq_4[key]['item']['Sequence']
    striped_seq = seq.strip('X')
    if len(striped_seq) - len(striped_seq.replace('X', '')) == 3:
        striped_seq = seq.strip('X').replace('X', 'A')
        leading_x = len(seq) - len(seq.lstrip('X'))
        leading_x = None if leading_x == 0 else leading_x
        trailing_x = len(seq) - len(seq.rstrip('X'))
        trailing_x = None if trailing_x == 0 else -trailing_x
        if leading_x or trailing_x:
            striped_ss = dropped_seq_4[key]['item']['DSSP'][leading_x:trailing_x]
            if len(striped_seq) == len(striped_ss):
                seq = ("*" * padding) + striped_seq + ("*" * padding)
                ss_seq = ("*" * padding) + striped_ss + ("*" * padding)
                try:
                    values = np.array([aa_keys.index(aa) for aa in seq], dtype=np.int8).reshape((-1, 1))
                    values = np.append(values, np.array([ss_keys.index(ss) for ss in ss_seq], dtype=np.int8).reshape((-1, 1)), axis=1)
                    values = np.append(values, np.array([get_psiphi(item["PsiPhi"], aa, i-padding) for i, aa in enumerate(seq)]).reshape((-1, 2)), axis=1)
                    new_data[key] = values
                except ValueError:
                    dropped_seq_5[key] = {"item": dropped_seq_4[key]['item'], "error": "non-cononical aa"}
                    continue
        
    else:
        dropped_seq_5[key] = {"item": dropped_seq_4[key]['item'], "error": "internal X more than 3"}
write_json(new_data, str(new_wd / "fixed_with_three_internal_X.json.gz"), encoder='numpy')

In [78]:
print(len(dropped_seq_5.keys()))

89


In [79]:
try_again = [key for key, item in dropped_seq_5.items() if item['error'] == "internal X more than 3"]
len(try_again)

89

In [30]:
def get_psiphi(d, aa, i): 
    try:
        return d[f"{aa_dict[aa]}-{i+1}"]
    except KeyError:
        return [np.nan, np.nan]
    
dropped_seq = {}
dropped_seq_domain = {}
        
for file in files[:1]:
    print(f'transforming {file}')
    filename = file.split('/')[-1]
    data = read_json(file)
    keys = list(data.keys())
    new_data = {}
    pfam_domains = {}
    aa_keys = "WYFMILVAGPSTCEDQNHRK*"
    ss_keys = "LSTIGHBE*"
    w = 20
    padding = w - 1
    i = 0
    for key in keys:
        item = data[key]
        pdbID = key.split('_')[0]
        try:
            pfam_domains[key] = prody.searchPfam(pdbID)
            pdb_file = f'{pdbID}.pdb.gz'
            system(f'mv {pdb_file} {pdb_files/pdb_file}')
        except:
            dropped_seq_domain[key] = "pfam failed"
        break
#         if len(item["Sequence"]) < 50:
#             dropped_seq[key] = {"item": item, "error": "shorter than 50 aa"}
#             continue
#         seq = ("*" * padding) + item["Sequence"] + ("*" * padding)
#         ss_seq = ("*" * padding) + item["DSSP"] + ("*" * padding)
#         assert len(seq) == len(ss_seq), f"{key} failed to transform, seq and ss have different lengths."
#         length = len(seq)
#         try:
#             values = np.array([aa_keys.index(aa) for aa in seq], dtype=np.int8).reshape((-1, 1))
#             values = np.append(values, np.array([ss_keys.index(ss) for ss in ss_seq], dtype=np.int8).reshape((-1, 1)), axis=1)
#             values = np.append(values, np.array([get_psiphi(item["PsiPhi"], aa, i-padding) for i, aa in enumerate(seq)]).reshape((-1, 2)), axis=1)
#             new_data[key] = values
#         except ValueError:
#             dropped_seq[key] = {"item": item, "error": "non-cononical aa"}
#             continue
        

        # store domain names
        
        ##
        
        
    write_json(new_data, str(new_wd / filename), encoder='numpy')

{'200L_1_A': 'pfam failed'}